In [1]:
import os

In [2]:
%pwd

'/Users/pparashar21/Desktop/Projects/MLOps/CancerDetectionMLOps/research'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_mode: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    params_name: str

In [ ]:
from CancerClassification.constants import *
from CancerClassification.utils.utility import read_yaml, create_directories 

In [15]:
class configManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path= Path(config.base_model_path),
            updated_base_mode= Path(config.updated_base_mode),
            params_image_size= self.config.params_image_size,
            params_learning_rate= self.config.params_learning_rate,
            params_include_top= self.config.params_include_top,
            params_weights= self.config.params_weights,
            params_classes= self.config.params_classes,
            params_name= self.config.params_name
        )

        return prepare_base_model_config 

In [16]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [17]:
class Prepare:
    def __init__(self, config:PrepareBaseModelConfig):
        self.config = config

    #download base model
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape= self.config.params_image_size,
            weights= self.config.params_weights,
            include_top= self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)
    
    @staticmethod
    def __prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers [:-freeze_till]:
                model.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units = classes,
            activation = "softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer = tf.keras.optimizers.SGD(Learning_rate=Learning_rate),
            loss = tf.keras.losses.CategoricalCrossentropy(),
            metrics = ['accuracy']
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self.__prepare_full_model(
            model=self.model,
            classes =self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate            
        )


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [ ]:
try:
    config=configManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config = prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e